In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from functions import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [5]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

Gradient Descent(0/49): loss=0.5, w0=3.229994853120003e-05, w1=-8.185259581200002e-06
Gradient Descent(1/49): loss=0.4684975020271792, w0=4.737735523736882e-05, w1=-8.288091556578185e-06
Gradient Descent(2/49): loss=0.4513712748460005, w0=7.2830273140742e-05, w1=-1.4153561994065591e-05
Gradient Descent(3/49): loss=0.44178360296424807, w0=8.891083213320669e-05, w1=-1.5810090997021512e-05
Gradient Descent(4/49): loss=0.43617479340076165, w0=0.00010996659128578852, w1=-2.044325562731981e-05
Gradient Descent(5/49): loss=0.4326884948512339, w0=0.0001257725187546787, w1=-2.2878720716603202e-05
Gradient Descent(6/49): loss=0.4303539841023341, w0=0.00014379601356697091, w1=-2.684601115822895e-05
Gradient Descent(7/49): loss=0.4286614226647361, w0=0.00015875071836384647, w1=-2.966032971941444e-05
Gradient Descent(8/49): loss=0.4273411475179239, w0=0.00017453053731563144, w1=-3.3257608571233524e-05
Gradient Descent(9/49): loss=0.4262489193466734, w0=0.00018840941370472445, w1=-3.624498374587162e

In [ ]:
# Least squares regression using normal equation
weight, loss = least_squares(y, tX)
loss

In [ ]:
# Ridge regression using normal equations
lambda_ = 1e-3
weight, loss = ridge_regression(y, tX, lambda_)
loss

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)